# Entrenar (fine-tuning) Spanish GPT-2 para generación de texto biblico
### Adaptación del [Colab](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/language_modeling_from_scratch.ipynb) de Sylvain Gugger

- **CLM**: el modelo tiene que predecir el siguiente token en la oración (por lo que las etiquetas son las mismas que las entradas desplazadas hacia la derecha). Para asegurarse de que el modelo no haga trampa, obtiene una máscara de atención que le impedirá acceder a los tokens después del token i cuando intente predecir el token i + 1 en la oración.

[Probar el modelo](https://huggingface.co/helenpy/spanish-gpt2-finetuned-rap-lyrics-finetuned-biblia)

### Comprobamos la GPU asignada

In [ ]:
! nvidia-smi

Fri Jun 23 12:32:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Autenticación en HF (para compartir modelo)

Para poder compartir nuestro modelo con la comunidad y generar resultados como el que se muestra en la imagen de arriba a través de la API de inferencia, hay algunos pasos más a seguir.

Primero debe almacenar su token de autenticación del sitio web HuggingFace (regístrate [aquí] (https://huggingface.co/join) si aún no lo ha hecho), luego ejecute la siguiente celda e ingrese su nombre de usuario y contraseña:

### Instalamos las dependencias requeridas

In [ ]:
!pip install datasets
!pip install transformers==4.28.0
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.9 MB/s eta 0:00

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


Luego, debemos instalar `Git-LFS` y configurar Git si aún no lo hemos hecho. Descomenta las siguientes instrucciones e introduzca su nombre y correo electrónico:

In [ ]:
!apt install git-lfs
!git config --global user.email "vg055@hotmail.com"
!git config --global user.name "Victor055"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


### Elegimos un modelo (checkpoint)

 -spanish-gpt2-finetuned-rap-lyrics fue entrando con 9,322 letras de rap en español.

 -GPT2-small-spanish fue entrenado con wikipedia en español utilizando el aprendizaje por transferencia y técnicas de ajuste fino. El entrenamiento duro aproximadamente 70 horas y se procesarón cerca de 3 GB de datos de entrenamiento.


In [ ]:
model_checkpoint = "Narrativa/spanish-gpt2-finetuned-rap-lyrics"
#helenpy/spanish-gpt2-finetuned-rap-lyrics-finetuned-biblia
#model_checkpoint = "datificate/gpt2-small-spanish"

### Elegimos y pre-procesamos el dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset, load_dataset, DatasetDict
import pandas as pd
#Lectura del archivo donde se encuentran los datos de entrenamiento
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tutoriales_MeIA/corpusTASS-2020/train.tsv',sep='\t')
#Unicamente utilizamos el texto y las demás columnas se eliminan
data = data.drop(columns=['id','etiqueta','pais'])
print(data.head(5),"\n")

#Se convierte a un datasetDict
ds_train = Dataset.from_pandas(data)
ds_dict = {'train' : ds_train}
dataset = DatasetDict(ds_dict)
dataset

                                               texto
0  @morbosaborealis jajajaja... eso es verdad... ...
1  @Adriansoler espero y deseo que el interior te...
2  comprendo que te molen mis tattoos, pero no te...
3  Mi última partida jugada, con Sona support. La...
4  Tranquilos que con el.dinero de Camacho seguro... 



DatasetDict({
    train: Dataset({
        features: ['texto'],
        num_rows: 4802
    })
})

In [ ]:
dataset["train"][1:10]

{'texto': ['@Adriansoler espero y deseo que el interior te cause lo mismo ',
  "comprendo que te molen mis tattoos, pero no te los hagas 'iguales' ",
  'Mi última partida jugada, con Sona support. La grandes razones para jugar Sona https://t.co/2rgmMeAFIs',
  'Tranquilos que con el.dinero de Camacho seguro mañana empiezan las escabadoras en el Arraijanal ',
  '@daniacal aún no, pero si estará jugable en el TGS no creo que tarde demasiado. ',
  '@ragnomuelle Yo a veces hecho de menos mi pelo largo por eso mismo entre otras cosas ',
  'A mí nunca me podrán hacer una broma porque no cojo llamadas y menos cuando son ocultas ',
  '#feliz septiembre..es bonito retarse..es increible lo mucho que puedes aprender.. medirse con el obstáculo..eres la joyita de la corona',
  'Este año el Madrid hará triplete y si lo hace me rapo al cero  grabad el mensaje jejej #ChiringuitoPiqué']}

### Tokenización del dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenize_fn(examples):
    return tokenizer(examples["texto"])

In [ ]:
tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=["texto"])
tokenized_dataset

Map:   0%|          | 0/4802 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4802
    })
})

In [ ]:
tokenized_dataset["train"][1]

{'input_ids': [36,
  3514,
  325,
  8794,
  348,
  269,
  5530,
  297,
  4926,
  293,
  295,
  3427,
  489,
  37808,
  362,
  951,
  225],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
len(tokenized_dataset["train"][1]['input_ids'])

17

### Formatear el dataset

Ahora, para la parte más difícil: necesitamos concatenar todos nuestros textos juntos y luego dividir el resultado en pequeños trozos de un cierto `block_size`. Para hacer esto, usaremos el método `map` nuevamente, con la opción` batched = True`. Esta opción en realidad nos permite cambiar la cantidad de ejemplos en los conjuntos de datos al devolver una cantidad diferente de ejemplos a la que obtuvimos. De esta manera, podemos crear nuestras nuevas muestras a partir de un lote de ejemplos.

Primero, tomamos la longitud máxima con la que se entrenó previamente nuestro modelo. Esto podría ser demasiado grande para caber en la RAM de nuestra GPU, por lo que aquí tomamos un poco menos de solo 128.

In [ ]:
#tokenizer.model_max_length
block_size = 128
block_size

128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k:sum(examples[k],[]) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000)

lm_datasets

Map:   0%|          | 0/4802 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 869
    })
})

Primero tenga en cuenta que duplicamos las entradas para nuestras etiquetas. Esto se debe a que el modelo de la biblioteca 🤗 Transformers aplica el desplazamiento a la derecha, por lo que no es necesario que lo hagamos manualmente.

También tenga en cuenta que, por defecto, el método `map` enviará un lote de 1000 ejemplos para ser tratados por la función de preprocesamiento. Entonces, aquí, soltaremos el resto para hacer que los textos tokenizados concatenados sean un múltiplo de `block_size` cada 1,000 ejemplos. Puede ajustar este comportamiento pasando un tamaño de lote mayor (que también se procesará más lentamente).

Ejemplo de una instancia después del pre-procesamiento

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' jugable en el TGS no creo que tarde demasiado. @ragnomuelle Yo a veces hecho de menos mi pelo largo por eso mismo entre otras cosas A mí nunca me podrán hacer una broma porque no cojo llamadas y menos cuando son ocultas #feliz septiembre..es bonito retarse..es increible lo mucho que puedes aprender.. medirse con el obstáculo..eres la joyita de la coronaEste año el Madrid hará triplete y si lo hace me rapo al cero  grabad el mensaje jejej #ChiringuitoPiquéEs conveniente tener un Gobierno democrático que cumpla su función politica. NO lo'

Dividir dataset en conjuntos de entrenamiento y prueba

In [ ]:
train_size = 800
test_size = 69

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 69
    })
})

### Entrenar el modelo

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import Trainer, TrainingArguments
import accelerate

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "gpt2-TASS2020-clm",
    #overwrite_output_dir=True,
    #per_device_train_batch_size = 2,
    #per_device_eval_batch_size = 2,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    hub_model_id=f"{model_name}-finetuned-TASS2020"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
)

Cloning https://huggingface.co/vg055/spanish-gpt2-finetuned-rap-lyrics-finetuned-TASS2020 into local empty directory.


In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 402.83


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,5.369411
2,No log,5.302528
3,No log,5.286397


TrainOutput(global_step=300, training_loss=5.337086588541666, metrics={'train_runtime': 94.24, 'train_samples_per_second': 25.467, 'train_steps_per_second': 3.183, 'total_flos': 156775219200000.0, 'train_loss': 5.337086588541666, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 197.63


In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/487M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.56k [00:00<?, ?B/s]

Upload file runs/Jun23_13-26-40_c5479f95fbd9/events.out.tfevents.1687526995.c5479f95fbd9.610.2:   0%|         …

Upload file runs/Jun23_13-26-40_c5479f95fbd9/1687526881.0155332/events.out.tfevents.1687526881.c5479f95fbd9.61…

Upload file runs/Jun23_13-26-40_c5479f95fbd9/events.out.tfevents.1687526876.c5479f95fbd9.610.0:   0%|         …

To https://huggingface.co/vg055/spanish-gpt2-finetuned-rap-lyrics-finetuned-TASS2020
   ea61d2a..1c62441  main -> main

   ea61d2a..1c62441  main -> main

To https://huggingface.co/vg055/spanish-gpt2-finetuned-rap-lyrics-finetuned-TASS2020
   1c62441..cb1e957  main -> main

   1c62441..cb1e957  main -> main



'https://huggingface.co/vg055/spanish-gpt2-finetuned-rap-lyrics-finetuned-TASS2020/commit/1c62441e8d72948ebdbbd8081eea30054a1da813'

In [ ]:
tokenizer.push_to_hub(f"{model_name}-finetuned-TASS2020")

CommitInfo(commit_url='https://huggingface.co/vg055/spanish-gpt2-finetuned-rap-lyrics-finetuned-TASS2020/commit/f9669b521f77f18941465d27bfbefd044e96950b', commit_message='Upload tokenizer', commit_description='', oid='f9669b521f77f18941465d27bfbefd044e96950b', pr_url=None, pr_revision=None, pr_num=None)

**Guarda modelo y utilizarlo desde disco local**

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/Tutoriales_MeIA")

In [ ]:
from transformers import AutoModelForCausalLM
model2 = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Tutoriales_MeIA/")

In [ ]:
from transformers import pipeline
tx = pipeline("text-generation", model=model2, tokenizer=tokenizer)
texto = "Hoy el mundo maltrata la naturleza"
tx(texto)

EJEMPLOS

In [ ]:
from transformers import pipeline

text_generation_1 = pipeline("text-generation", model="Narrativa/spanish-gpt2-finetuned-rap-lyrics")
text_generation_2 = pipeline("text-generation", model="vg055/spanish-gpt2-finetuned-rap-lyrics-finetuned-TASS2020")

In [ ]:
texto = "Déjame contarte lo importante que es buscarte un plan No para golpearles o ganarles, sino para darles paz"
t1=text_generation_1(texto)
t2=text_generation_2(texto)
print(t1)
print(t2)


[{'generated_text': 'Déjame contarte lo importante que es buscarte un plan No para golpearles o ganarles, sino para darles paz,\nun gesto amable sin pensar que tal vez se acabe el mundo, no, no, no, noDejemos todo en la mesa y sigamos'}]
[{'generated_text': 'Déjame contarte lo importante que es buscarte un plan No para golpearles o ganarles, sino para darles paz. (R-A-B-N-A-A) Agradece a tus hijos y a todas las personas que son parte de'}]
